# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [2]:
import numpy as np
import pandas as pd

In [3]:
in_filepath = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

wing_df = pd.read_csv(in_filepath + points_filename)
norm_df = pd.read_csv(in_filepath + norms_filename)

print(wing_df)
print(norm_df)

              x         y         z
0      0.440148  0.073950  0.103123
1      0.713695  0.209429  0.055195
2      0.451790  0.021569  0.052462
3      0.032607  0.154912  0.108069
4      0.750952  0.139930  0.113273
...         ...       ...       ...
19995  0.913177  0.282509  0.051195
19996  0.115440  0.221203  0.028832
19997  0.453917  0.034118  0.052462
19998  0.556022  0.063779  0.101915
19999  0.030382  0.130567  0.011402

[20000 rows x 3 columns]
              x         y         z
0     -1.000000 -0.000293 -0.000429
1     -0.283259 -0.649111 -0.705988
2      0.000000  0.000000 -1.000000
3     -0.974604  0.223384  0.015692
4     -0.325352 -0.933390  0.151428
...         ...       ...       ...
19995 -0.018169 -0.132277  0.991046
19996 -0.991501 -0.031098 -0.126326
19997  0.000000  0.000000 -1.000000
19998  1.000000 -0.000293 -0.000429
19999 -0.730616  0.157085 -0.664473

[20000 rows x 3 columns]


In [4]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [5]:
class PINN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_units):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        in_units = input_dim
        for units in hidden_units:
            layer = nn.Linear(in_units, units)
            nn.init.xavier_normal_(layer.weight)  # Apply Xavier initialization
            self.layers.append(layer)
            in_units = units
        output_layer = nn.Linear(in_units, output_dim)
        nn.init.xavier_normal_(output_layer.weight)  # Apply Xavier initialization
        self.layers.append(output_layer)

    def forward(self, input):
        for layer in self.layers[:-1]:
            output = torch.tanh(layer(input))
            input = output
        output = self.layers[-1](input)
        return output

    def loss(self, x_f, y_f, z_f, t_f, x0, y0, z0, t0, x_b, y_b, z_b, t_b,
             mu, rho, dt, c1, c2, c3, c4):

        xyzt_combinations = torch.cartesian_prod(x_f.flatten(), y_f.flatten(), z_f.flatten(), t_f.flatten())
        output = self(xyzt_combinations)
        u = output[:, 0]
        v = output[:, 1]
        w = output[:, 2]
        p = output[:, 3]

        u_t = torch.autograd.grad(u, t_f, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_x = torch.autograd.grad(u, x_f, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_y = torch.autograd.grad(u, y_f, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_z = torch.autograd.grad(u, z_f, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_xx = torch.autograd.grad(u_x, x_f, grad_outputs=torch.ones_like(u_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_yy = torch.autograd.grad(u_y, y_f, grad_outputs=torch.ones_like(u_y), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_zz = torch.autograd.grad(u_z, z_f, grad_outputs=torch.ones_like(u_z), create_graph=True, retain_graph=True, only_inputs=True)[0]

        v_t = torch.autograd.grad(v, t_f, grad_outputs=torch.ones_like(v), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_x = torch.autograd.grad(v, x_f, grad_outputs=torch.ones_like(v), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_y = torch.autograd.grad(v, y_f, grad_outputs=torch.ones_like(v), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_z = torch.autograd.grad(v, z_f, grad_outputs=torch.ones_like(v), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_xx = torch.autograd.grad(v_x, x_f, grad_outputs=torch.ones_like(v_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_yy = torch.autograd.grad(v_y, y_f, grad_outputs=torch.ones_like(v_y), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_zz = torch.autograd.grad(v_z, z_f, grad_outputs=torch.ones_like(v_z), create_graph=True, retain_graph=True, only_inputs=True)[0]

        w_t = torch.autograd.grad(w, t_f, grad_outputs=torch.ones_like(w), create_graph=True, retain_graph=True, only_inputs=True)[0]
        w_x = torch.autograd.grad(w, x_f, grad_outputs=torch.ones_like(w), create_graph=True, retain_graph=True, only_inputs=True)[0]
        w_y = torch.autograd.grad(w, y_f, grad_outputs=torch.ones_like(w), create_graph=True, retain_graph=True, only_inputs=True)[0]
        w_z = torch.autograd.grad(w, z_f, grad_outputs=torch.ones_like(w), create_graph=True, retain_graph=True, only_inputs=True)[0]
        w_xx = torch.autograd.grad(w_x, x_f, grad_outputs=torch.ones_like(w_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
        w_yy = torch.autograd.grad(w_y, y_f, grad_outputs=torch.ones_like(w_y), create_graph=True, retain_graph=True, only_inputs=True)[0]
        w_zz = torch.autograd.grad(w_z, z_f, grad_outputs=torch.ones_like(w_z), create_graph=True, retain_graph=True, only_inputs=True)[0]

        p_x = torch.autograd.grad(p, x_f, grad_outputs=torch.ones_like(p), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_xx = torch.autograd.grad(p_x, x_f, grad_outputs=torch.ones_like(p_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_y = torch.autograd.grad(p, y_f, grad_outputs=torch.ones_like(p), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_yy = torch.autograd.grad(p_y, y_f, grad_outputs=torch.ones_like(p_y), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_z = torch.autograd.grad(p, z_f, grad_outputs=torch.ones_like(p), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_zz = torch.autograd.grad(p_z, z_f, grad_outputs=torch.ones_like(p_z), create_graph=True, retain_graph=True, only_inputs=True)[0]

        # b = rho * ( 1/dt * (u_x + v_y) - u_x**2 - 2*u_y*v_x - v_y**2)

        f1 = u_t + u*u_x + v*u_y + w*u_z + (1/rho) * p_x - mu * (u_xx + u_yy + u_zz)
        f2 = v_t + u*v_x + v*v_y + w*v_z + (1/rho) * p_y - mu * (v_xx + v_yy + v_zz)
        f3 = w_t + u*w_x + v*w_y + w*w_z + (1/rho) * p_z - mu * (w_xx + w_yy + w_zz)
        f3 = u_x + v_y + w_z
        # TODO: add poisson equation & impermeability condition
        # f4 = p_xx + p_yy + p_zz - b

        # Initial condition loss
        output_init = self(torch.cat([x0, y0, z0, t0], dim=1))
        u0_pred = output_init[:, 0]
        v0_pred = output_init[:, 1]
        w0_pred = output_init[:, 2]
        p0_pred = output_init[:, 3]

        # for x > 0 and t = 0 -> u, v, p = 0

        u0_true = torch.zeros_like(u0_pred)
        v0_true = torch.zeros_like(v0_pred)
        w0_true = torch.zeros_like(w0_pred)
        p0_true = torch.ones_like(p0_pred)

        ic_loss_u = torch.mean(torch.square(u0_pred - u0_true))
        ic_loss_v = torch.mean(torch.square(v0_pred - v0_true))
        ic_loss_w = torch.mean(torch.square(w0_pred - w0_true))
        ic_loss_p = torch.mean(torch.square(p0_pred - p0_true))

        # Boundary conditions loss

        xyzt_combinations = torch.cartesian_prod(x_b.flatten(), y_b.flatten(), z_b.flatten(), t_b.flatten())
        output_boundary = self(xyt_combinations)
        u_b_pred = output_boundary[:, 0]
        v_b_pred = output_boundary[:, 1]
        w_b_pred = output_boundary[:, 2]

        # u = 1, v = 0 and w = 0 for x = 0

        u_b_true = torch.ones_like(u_b_pred)
        v_b_true = torch.zeros_like(v_b_pred)
        w_b_true = torch.zeros_like(w_b_pred)
        
        bc_loss_u = torch.mean(torch.square(u_b_pred - u_b_true))
        bc_loss_v = torch.mean(torch.square(v_b_pred - v_b_true))
        bc_loss_w = torch.mean(torch.square(w_b_pred - w_b_true))


        # TODO: wing surface boundary conditions loss
        # ## down
        # xyt_combinations = torch.cartesian_prod(x_box_down.flatten(), y_box_down.flatten(), t_box.flatten())
        # output_boundary_box_down = self(xyt_combinations)
        # u_box_down_pred = output_boundary_box_down[:, 0]
        # v_box_down_pred = output_boundary_box_down[:, 1]

        # u_box_down_true = torch.ones_like(u_box_down_pred)
        # v_box_down_true = torch.zeros_like(v_box_down_pred)
        
        # box_down_loss_u = torch.mean(torch.square(u_box_down_pred - u_box_down_true))
        # box_down_loss_v = torch.mean(torch.square(v_box_down_pred - v_box_down_true))

        # Combine PDE residual, initial condition, and boundary condition losses
        pde_loss =  torch.mean(torch.square(f1)) + \
                    torch.mean(torch.square(f2)) + \
                    torch.mean(torch.square(f3)) / 3
        
        ic_loss = (ic_loss_u + ic_loss_v + ic_loss_p) / 3 
        
        bc_loss = (bc_loss_u + bc_loss_v) / 2

        # bc_box_loss = (box_left_loss_u + box_left_loss_v + \
        #                 box_up_loss_u + box_up_loss_v + \
        #                 box_right_loss_u + box_right_loss_v + \
        #                 box_down_loss_u + box_down_loss_v) / 8
        
        # bc_box_loss = torch.tensor(0)

        # bc_box_loss = (box_left_loss_u + box_left_loss_v) / 2

        total_loss =    c1 * pde_loss + \
                        c2 * ic_loss + \
                        c3 * bc_loss
                        # c4 * bc_box_loss

        return total_loss, pde_loss, ic_loss, bc_loss

In [6]:
def random_uniform_domains(n_samples, domains):
    samples = np.concatenate([np.random.uniform(low, high, n_samples) for low, high in domains])
    return samples.reshape(-1, 1)

def domains_union(domains, dx, reshape=True):
    samples = np.concatenate([np.arange(low, high, dx) for low, high in domains])
    
    if reshape:
        return samples.reshape(-1, 1)
    else:
        return samples

In [7]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 30
Ny = 30
Nz = 30
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
# x_test = domains_union([(0, 0.3), (0.7, x_max)], dx)
y_test = np.linspace(0, y_max, Ny)
# y_test = domains_union([(0, 0.3), (0.7, y_max)], dy)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_gripd, t_grid = np.meshgrid(x_test, y_test, z_test, t_test)

In [8]:
input_dim = 4
output_dim = 4
# hidden_units = [32, 32, 32]
# hidden_units = [64, 64, 64, 64]
# hidden_units = [128, 128, 128, 128]
hidden_units = [256, 256, 256, 256]
# hidden_units = [512, 512]
# hidden_units = [1024, 1024, 1024]
# hidden_units = [20, 40, 80, 100, 100, 80, 40, 20]

if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print (x)
else:
    print ("MPS device not found.")
    device = "cpu"

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

NameError: name 'mps_device' is not defined

In [ ]:
# optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)
optimizer = torch.optim.LBFGS(pinn.parameters())

epochs = 1000
# Nf = int(Nx / 10)   # num of collocation points -> pde evaluation -> Nf^3... needs fixing: sample Nf points from the whole 3D domain
Nf = Nx   # num of collocation points -> pde evaluation -> Nf^3... needs fixing: sample Nf points from the whole 3D domain
N0 = Ny    # num of points to evaluate initial conditons
# Nb = int(Nx / 10)    # num of points to evaluate boundary conditions -> Nb^3... needs fixing: sample Nf points from the whole 3D domain
Nb = Nx    # num of points to evaluate boundary conditions -> Nb^3... needs fixing: sample Nf points from the whole 3D domain
# Nbox = 30  # num of points to evaluate boundary condition at each edge of the box

# Density (rho): 1.184 kg/m³
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1184
mu = 1.81e-5

In [ ]:
def closure(report_losses=False):
    optimizer.zero_grad()

    x_f = torch.tensor(random_uniform_domains(Nf, [(0, x_max)]), dtype=torch.float32, device=device, requires_grad=True) # collocation points
    y_f = torch.tensor(random_uniform_domains(Nf, [(0, y_max)]), dtype=torch.float32, device=device, requires_grad=True) # collocation points
    z_f = torch.tensor(random_uniform_domains(Nf, [(0, z_max)]), dtype=torch.float32, device=device, requires_grad=True) # collocation points
    t_f = torch.tensor(np.random.uniform(0, t_max, size=(Nf, 1)), dtype=torch.float32, device=device, requires_grad=True) # collocation points

    # TODO investigate
    x0 = torch.tensor(np.random.uniform(dx, x_max, size=(N0, 1)), dtype=torch.float32, device=device, requires_grad=True)
    y0 = torch.tensor(np.random.uniform(dy, y_max, size=(N0, 1)), dtype=torch.float32, device=device, requires_grad=True) 
    z0 = torch.tensor(np.random.uniform(dz, z_max, size=(N0, 1)), dtype=torch.float32, device=device, requires_grad=True) 
    t0 = torch.zeros_like(x0)
    # t0 = torch.zeros(domains_union([(dt, t_max)], dt), dtype=torch.float32, device=device, requires_grad=True)

    x_b = torch.zeros(size=(Nb, 1), dtype=torch.float32, device=device, requires_grad=True)
    y_b = torch.tensor(np.random.uniform(0, y_max, size=(Nb, 1)), dtype=torch.float32, device=device, requires_grad=True)
    z_b = torch.tensor(np.random.uniform(0, z_max, size=(Nb, 1)), dtype=torch.float32, device=device, requires_grad=True)
    t_b = torch.tensor(np.random.uniform(0, t_max, size=(Nb, 1)), dtype=torch.float32, device=device, requires_grad=True)

    total_loss, pde_loss, ic_loss, bc_loss = pinn.loss(
                    x_f, y_f, z_f, t_f, x0, y0, z0, t0, x_b, y_b, z_b, t_b, 
                    mu, rho, dt, 0.005, 0.25, 0.25, 0.25)

    total_loss.backward()

    if report_losses:
        return total_loss, pde_loss, ic_loss, bc_loss
    else:
        return total_loss

for epoch in range(epochs):
    optimizer.step(closure)
    if epoch % 10 == 0:
        total_loss, pde_loss, ic_loss, bc_loss = closure(report_losses=True)
        print(f'Epoch: {epoch},\tTotal loss: {total_loss.item()},\tPDE loss: {pde_loss.item()},\tIC loss: {ic_loss.item()},\tBC loss: {bc_loss.item()}')


: 